In [1]:
#Kütüphanelerimizi yüklüyoruz
import os
import cv2
import numpy as np

from metods import fillHoles, readImages, removeSmallObjects
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from matplotlib import pyplot as plt, image
from PIL import Image
import numpy as np


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [6]:
def SaveGeneratedMask(Image_collection_path, save_path, modelName, IMG_HEIGHT = 128, IMG_WIDTH = 128):
    test_images = os.listdir(Image_collection_path)
    x = np.zeros((len(test_images), IMG_HEIGHT, IMG_WIDTH))
    
    for i, img_id in enumerate(test_images):
        img_collection = Image_collection_path  + img_id
        img = readImages(img_collection,  IMG_HEIGHT = 128, IMG_WIDTH = 128, is_gray=0)
        x[i] = img
    
    model = load_model(modelName)
    pred_img = model.predict(x/255.)*255.0
    
    for j, img_id in enumerate(test_images):   
        new_img_path =save_path + 'mask_' + img_id
        
        #Oluşturulan maskelerimize görüntü işleme adımlarını uyguluyoruz
        img = fillHoles(pred_img[j])
        img = removeSmallObjects(img, min_area = 200)
        
        try:
            if img == None:
                continue
        except:
            pass
        
        image.imsave(new_img_path, img, cmap=plt.cm.gray)
        print(f"[{j + 1}/{len(test_images)}] Maske oluşturuldu.")

In [3]:
def saveSegmentedLung(maskPath, LungPath, savePath, IMG_HEIGHT = 128, IMG_WIDTH = 128):
    mask = readImages(maskPath,  IMG_HEIGHT = IMG_HEIGHT, IMG_WIDTH = IMG_WIDTH, is_gray=0)
    _ , mask = cv2.threshold(mask, 120, 255, cv2.THRESH_BINARY) #binary img
    img = readImages(LungPath,  IMG_HEIGHT = IMG_HEIGHT, IMG_WIDTH = IMG_WIDTH, is_gray=0)
    segmented_img = cv2.bitwise_and(img, img, mask=mask)
    image.imsave(savePath, segmented_img, cmap=plt.cm.bone)

In [4]:
#Dosya yollarını tanımlıyoruz.
healty_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Healty\\"
healty_mask_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Healty Mask\\"
healty_segmented_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Healty Segmented\\"

pneu_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Pneumothorax\\"
pneu_mask_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Pneumothorax Mask\\"
pneu_segmented_path = "C:\\Users\\ugur_\\Python Projects\\LungClassifier\\NIH Data\\Pneumothorax Segmented\\"

In [ ]:
#Normal X-ray görüntülerinin maskelerini oluşturuyoruz.
SaveGeneratedMask(Image_collection_path=healty_path, save_path=healty_mask_path, modelName='model.h5', IMG_HEIGHT=128, IMG_WIDTH=128)

In [ ]:
#Pnömotoraks X-ray görüntülerinin maskelerini oluşturuyoruz.
SaveGeneratedMask(Image_collection_path=pneu_path, save_path=pneu_mask_path, modelName='model.h5', IMG_HEIGHT=128, IMG_WIDTH=128)

In [18]:
#Sağlıklı göğüs X-ray'lerinden ciğer bölümünü maskemizi kullanarak segmentliyoruz.
for img_id in os.listdir(healty_mask_path):
    mask_file = healty_mask_path + img_id
    img_file = healty_path + img_id.replace("mask_", "")
    segmented_file = healty_segmented_path + "segment_" + img_id
    try:
        saveSegmentedLung(mask_file, img_file, segmented_file, IMG_HEIGHT = 128, IMG_WIDTH = 128)
    except Exception as ex:
        print("HATA: ", ex)
        continue


In [19]:
#Pnömotoraks göğüs X-ray'lerinden ciğer bölümünü maskemizi kullanarak segmentliyoruz.
for img_id in os.listdir(pneu_mask_path):
    mask_file = pneu_mask_path + img_id
    img_file = pneu_path + img_id.replace("mask_", "")
    segmented_file = pneu_segmented_path + "segment_" + img_id
    try:
        saveSegmentedLung(mask_file, img_file, segmented_file, IMG_HEIGHT = 128, IMG_WIDTH = 128)
    except Exception as ex:
        print("HATA: ", ex)
        continue

In [20]:
#Kayıt sonrası oluşan siyah görselleri kontrol edip varsa siliyoruz.
for img_id in os.listdir(healty_mask_path):
    img_file = healty_mask_path + img_id
    img = Image.open(img_file)
    test_img = img.convert("L").getextrema()
    if test_img == (0, 0):
        print("Black image found!")
        print(img_id)
        os.remove(img_file)
        print("Deleted!")
        
for img_id in os.listdir(pneu_mask_path):
    img_file = pneu_mask_path + img_id
    img = Image.open(img_file)
    test_img = img.convert("L").getextrema()
    if test_img == (0, 0):
        print("Black image found!")
        print(img_id)
        os.remove(img_file)
        print("Deleted!")